In [206]:
import mlflow
from mlflow.models import infer_signature
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
import pickle
import pandas as pd
import numpy as np

In [189]:
mlflow.tracking.get_tracking_uri()
mlflow.set_tracking_uri(uri="http://localhost:8080")


<Experiment: artifact_location='mlflow-artifacts:/350600354360964916', creation_time=1729334647926, experiment_id='350600354360964916', last_update_time=1729334647926, lifecycle_stage='active', name='MLflow Quickstart', tags={'mlflow.sharedViewState.105fff6332944f66474d0a12178ab2f35b8d9038092d2e8dc4fe585440c38fa3': 'deflate;eJytU9tuGjEQ/RXkZ1RBkwpl3xZKUlSoqoVGlVAExh5gJK+98oWwQfx7x7tctkRRX/o445lzzpwZH5gDbsX2EZUHyxLG2sxYCbZffoeSYu69xVXw4D45z61feMzhWpQ6wZI1Vw7arHqfxeeEpeMxFSlcgyiFggt8KjzuYr/knjvw7vwyf2mz3EhQz2AdGn3tUKqVBe2ox4EC4UEOjAo5ZZJ5U95yaoIVsKTCZnYSQd1t9mtNv2REawl9uC+4liBZcjjWmZ+odYznp4pvKCXo2ziik8jHUTadLbqdRfbrx5SohMkLboF0D7ZkSpR6YCEg4bFu7/PD3V2vc999uP8i7Xq1WyFSjy+LCNVPM1YzDEzQfmYGNRZLuh2yCGgGMSUbyKPROzgOyHvcm2gwmeXjAKftoHsCDZZXOW8DnMFOexYiWC7KlhOWHV+aI5zYPh6iwap5tf7K9FaNHzcn0RWKl2didBlUB9TQ90+9x/YHFvZE9y3nr1f2aek85P+bnjzZIbxO+B5zfGuU0q7G6Pz5QC4AqRDEQs69ZwsOnqwJBchnrgK40eVOaoVblDDMC1/+nd7Enj7NoYNSp/DmdHnwJoO1Bbcdar5SDdKNMiuuxqihQh0YvcYNdbF9ukdXnwEZV1QfV8M0N8ZvNTii71w/3+9YPGlcDjse/w

In [ ]:
# Creating an experiment
experiment_id = mlflow.create_experiment(name="Mtest")

#To record runs to this experiment
mlflow.set_experiment("MLflow Quickstart")

#delete
mlflow.delete_experiment(experiment_id=experiment_id):

In [165]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pickle

class IrisExample:
    def __init__(self):
        self.X_train = None
        self.X_test = None
        self.y_test = None
        self.params = None
        print("Iris class initialized")

    # Train and save the model
    def train_model(self):
        # Load dataset
        data = load_iris()
        X, y = data.data, data.target

        self.X_train, self.X_test, y_train,  self.y_test = train_test_split(X, y, test_size=0.2, random_state=42
                                                           )
        # Define the model hyperparameters
        self.params = {
            "solver": "lbfgs",
            "max_iter": 2000,
            "multi_class": "auto",
            "random_state": 3232,
            }

        # Train model
        model = LogisticRegression(**self.params)
        model.fit(self.X_train, y_train)

        # Save the trained model
        with open("model.pkl", "wb") as f:
            pickle.dump(model, f)
        print("Model trained and saved as model.pkl")
        
    # Load model 
    def load_model(self):
        #self.model = None
        # Load the saved model
        with open("model.pkl", "rb") as f:
            self.model = pickle.load(f)
        return self.model
    
    #make a prediction using predefined test data
    def predict(self, data):
        model = self.load_model()
        # Test data (sample input for prediction)
        #test_data = [5.1, 3.5, 1.4, 0.2]  # Example features
        prediction = model.predict(data)
        #print(f"Prediction for {test_data}: {int(prediction[0])}")
        return prediction

    # Calculate metrics
    def metrics(self):
        y_pred = self.predict(self.X_test)
        accuracy = accuracy_score(self.y_test, y_pred)
        return accuracy


In [166]:
iris = IrisExample()
model = iris.train_model()

Iris class initialized
Model trained and saved as model.pkl


C:\Users\EdwinVivekN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [211]:
mlflow.enable_system_metrics_logging() 

In [212]:
# Start an MLflow run
with mlflow.start_run(log_system_metrics=True) as run:
    print(run.info.run_id)
    mlflow.log_params(iris.params)
    mlflow.log_metric("accuracy scr", iris.metrics())

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(iris.X_train, iris.predict(iris.X_train))

    conda_env = {
    "channels": ["default"],
    "dependencies": ["pip"],
    "pip": ["mlflow", "cloudpickle==1.6.0"],
    "name": "mlflow-env"}

    # Specifying pip requirements
    pip_requirements = ["mlflow"]

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=iris.load_model(),
        artifact_path="iris_model_ex",
        signature=signature,  #to enforce the input format for models 
        input_example=iris.X_train,  #This will make sure that the input example will match our input signature.
        registered_model_name="tracking-quickstart",
        #conda_env= conda_env,   # to help future users reproduce your project
        #pip_requirements= pip_requirements #  to help future users reproduce your project
        
    )

2024/10/25 15:29:12 WARNING mlflow.system_metrics.system_metrics_monitor: Skip logging GPU metrics because creating `GPUMonitor` failed with error: Failed to initialize NVML, skip logging GPU metrics: NVML Shared Library Not Found.
2024/10/25 15:29:12 INFO mlflow.system_metrics.system_metrics_monitor: Started monitoring system metrics.


7f5ed0b61bef47b78d7c08a6783c51bb


Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/10/25 15:29:40 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 7
Created version '7' of model 'tracking-quickstart'.


2024/10/25 15:29:45 INFO mlflow.tracking._tracking_service.client: 🏃 View run flawless-skunk-744 at: http://localhost:8080/#/experiments/350600354360964916/runs/7f5ed0b61bef47b78d7c08a6783c51bb.
2024/10/25 15:29:45 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/350600354360964916.
2024/10/25 15:29:45 INFO mlflow.system_metrics.system_metrics_monitor: Stopping system metrics monitoring...
2024/10/25 15:29:45 INFO mlflow.system_metrics.system_metrics_monitor: Successfully terminated system metrics monitoring!


In [204]:
# Load the model back for predictions as a generic Python Function model
pyfunc_predictions = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = pyfunc_predictions.predict(iris.X_test)

iris_feature_names = load_iris().feature_names

result = pd.DataFrame(iris.X_test, columns=iris_feature_names)
result["actual_class"] = iris.y_test
result["predicted_class"] = predictions

result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


In [184]:
#returns a mlflow.entities.Run object corresponding to the currently active run
active = mlflow.active_run()
print(active)

<ActiveRun: >


In [185]:
#In order to access  attributes (parameters, metrics, etc.) , use the MlflowClient
client = mlflow.MlflowClient()
data = client.get_run(mlflow.active_run().info.run_id).data
print([data if data is not None else 0])

[<RunData: metrics={}, params={}, tags={'mlflow.runName': 'agreeable-rook-249',
 'mlflow.source.name': 'C:\\Users\\EdwinVivekN\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python312\\site-packages\\ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'EdwinVivekN'}>]


In [188]:
mlflow.end_run()

2024/10/21 00:24:17 INFO mlflow.tracking._tracking_service.client: 🏃 View run agreeable-rook-249 at: http://localhost:8080/#/experiments/350600354360964916/runs/3cd4cb0ecd62408083e6c6742f4ef5c2.
2024/10/21 00:24:17 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:8080/#/experiments/350600354360964916.


In [186]:
mlflow.last_active_run()

<ActiveRun: >

In [ ]:
#mlflow.log_metric() logs a single key-value metric. The value must always be a number. MLflow remembers the history of values for each metric.
#Use mlflow.log_metrics() to log multiple metrics at once.


In [ ]:
#mlflow.log_param() logs a single key-value param in the currently active run. The key and value are both strings. 
#Use mlflow.log_params() to log multiple params at once.
mlflow.log_param()


In [ ]:
mlflow.log_input()

In [ ]:
#mlflow.set_tag() sets a single key-value tag in the currently active run. The key and value are both strings. 
#Use mlflow.set_tags() to set multiple tags at once.

In [176]:
#mlflow.log_artifact() logs a local file or directory as an artifact, optionally taking an artifact_path to place it in within the run’s artifact 
#mlflow.log_artifacts() logs all the files in a given directory as artifacts, again taking an optional artifact_path.

In [183]:
#mlflow.get_artifact_uri() returns the URI that artifacts from the current run should be logged to.
mlflow.get_artifact_uri()

'mlflow-artifacts:/350600354360964916/3cd4cb0ecd62408083e6c6742f4ef5c2/artifacts'

In [190]:
# Inspecting the signature
print(signature)

inputs: 
  [Tensor('float64', (-1, 4))]
outputs: 
  [Tensor('int32', (-1,))]
params: 
  None



In [192]:
#save a model
mlflow.sklearn.save_model(
    sk_model= iris.load_model(),
    path = "model",
    input_example=iris.X_train
)

In [203]:
#Load saved model
model_uri_saved = "model"

# Loading our model as a scikit-learn model
sklearn_model = mlflow.sklearn.load_model(model_uri=model_uri_saved)

# Inference with the scikit-learn model
sklearn_predictions = sklearn_model.predict(iris.X_test)
X_test_df = pd.DataFrame(iris.X_test, columns=iris_feature_names)
pyfunc_predictions = sklearn_model.predict(X_test_df)

C:\Users\EdwinVivekN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [207]:
# Inspecting our predictions
print(sklearn_predictions, 
      "\n\n", 
      np.equal(pyfunc_predictions, sklearn_predictions).all())

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0] 

 False


<h3>Model Registry

In [222]:
from mlflow import MlflowClient

client = MlflowClient()
client.create_registered_model("new-reg-model") #creates an empty registered model with no version associated

result = client.create_model_version(
    name = client.get_registered_model("new-reg-model").name,
    run_id= "7f5ed0b61bef47b78d7c08a6783c51bb",
    source = mlflow.get_artifact_uri("model")
)


RestException: RESOURCE_ALREADY_EXISTS: Registered Model (name=new-reg-model) already exists.

<h3>Building a Docker Image for MLflow Model

In [224]:
mlflow.models.build_docker(
    model_uri=f"runs:/7f5ed0b61bef47b78d7c08a6783c51bb/iris_model_ex",
    name="mlflow_docker_sample",
    enable_mlserver=True,
)

2024/11/03 15:59:34 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'


2024/11/03 15:59:39 INFO mlflow.pyfunc.backend: Building docker image with name mlflow_docker_sample


DockerException: Error while fetching server API version: (2, 'CreateFile', 'The system cannot find the file specified.')